In [172]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [173]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from deep_dss.helpers import *

In [174]:
data = split_count_and_lensing_maps_by_dataset("TRAINLITE", config="c", mixed_bias=True, rand_bias=True)
val = split_count_and_lensing_maps_by_dataset("TESTLITE", config="c", mixed_bias=True, rand_bias=True)

data["x"] = np.reshape(data["x"], (768, 262144, 1))
data["y"] = np.reshape(data["y"], (768, 1, 1))

val["x"] = np.reshape(data["x"], (768, 262144, 1))
val["y"] = np.reshape(data["y"], (768, 1, 1))

/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(
/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/healpy/fitsfunc.py:391: UserWarning: NSIDE = 1024
  warnings.warn("NSIDE = {0:d}".format(nside))
/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/healpy/fitsfunc.py:400: UserWarning: ORDERING = NESTED in fits file
  warnings.warn("ORDERING = {0:s} in fits file".format(ordering))
/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/healpy/fitsfunc.py:428: UserWarning: INDXSCHM = IMPLICIT
  warnings.warn("INDXSCHM = {0:s}".format(schm))


In [175]:
def build_model():
    return keras.Sequential([
        keras.Input(shape=(262144, 1)),
        layers.Conv1D(64, 4, strides=4, activation='relu'),
        layers.Conv1D(128, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(256, 4, strides=4, activation='relu'),
        layers.Conv1D(1, 4, strides=4, activation='relu'),
    ])

In [176]:
model = build_model()

In [177]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_103 (Conv1D)          (None, 65536, 64)         320       
_________________________________________________________________
conv1d_104 (Conv1D)          (None, 16384, 128)        32896     
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 4096, 256)         131328    
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 1024, 256)         262400    
_________________________________________________________________
conv1d_107 (Conv1D)          (None, 256, 256)          262400    
_________________________________________________________________
conv1d_108 (Conv1D)          (None, 64, 256)           262400    
_________________________________________________________________
conv1d_109 (Conv1D)          (None, 16, 256)         

In [178]:
model.compile(optimizer="adam", loss=tf.keras.losses.MAE, metrics=[])

In [179]:
logdir = "../log/v3-spherenn-simple-noisy-mixed-count-1"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

In [180]:
model.fit(x=data["x"], y=data["y"], batch_size=32, epochs=60, validation_data=(val["x"], val["y"]),
         callbacks=[tensorboard_callback])

Epoch 1/60
24/24 [==============================] - 56s 2s/step - loss: 0.3225 - val_loss: 0.1703
Epoch 2/60
24/24 [==============================] - 51s 2s/step - loss: 0.1841 - val_loss: 0.1739
Epoch 3/60
24/24 [==============================] - 51s 2s/step - loss: 0.1816 - val_loss: 0.1676
Epoch 4/60
24/24 [==============================] - 50s 2s/step - loss: 0.1757 - val_loss: 0.1645
Epoch 5/60
24/24 [==============================] - 50s 2s/step - loss: 0.1841 - val_loss: 0.1678
Epoch 6/60
24/24 [==============================] - 50s 2s/step - loss: 0.1787 - val_loss: 0.2191
Epoch 7/60
24/24 [==============================] - 50s 2s/step - loss: 0.1769 - val_loss: 0.1628
Epoch 8/60
24/24 [==============================] - 51s 2s/step - loss: 0.1674 - val_loss: 0.1634
Epoch 9/60
24/24 [==============================] - 51s 2s/step - loss: 0.1690 - val_loss: 0.1469
Epoch 10/60
24/24 [==============================] - 52s 2s/step - loss: 0.1590 - val_loss: 0.1332
Epoch 11/60
24/24 [

In [181]:
%tensorboard --logdir $logdir